# Well Placement and Control Optimization using a spatiotemporal proxy
### Misael M. Morales - 2024
***

In [586]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

from scipy.io import savemat, loadmat
from tqdm import tqdm

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.optimizers import *

sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz  = 100, 100, 11

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
Tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']

In [604]:
n_train = 250
n_steps = 10
t_steps = np.linspace(0, 25, n_steps, dtype=int)

xm = np.zeros((n_train,nx,ny,nz,2))
xw = np.zeros((n_train, 2, 5))
xc = np.zeros((n_train, n_steps, 5))
xt = np.zeros((n_train, n_steps, 1))
for i in tqdm(range(n_train)):
    d = np.load('data_npy_100_100_11/inputs_rock_rates_locs_time/x_{}.npz'.format(i))
    xm[i,...,0] = d['poro']
    xm[i,...,1] = d['perm']
    xw[i] = d['locs']
    xc[i] = d['ctrl'][t_steps]
    xt[i] = d['time'][t_steps]

xg = np.zeros((n_train,nx,ny,nz,2))
for i in range(n_train):
    xg[i,...,0] = Grid
    xg[i,...,1] = Tops



print('xm', xm.shape)
print('xg', xg.shape)
print('xw', xw.shape)
print('xc', xc.shape)
print('xt', xt.shape)


100%|██████████| 250/250 [00:00<00:00, 2469.51it/s]

xm (250, 100, 100, 11, 2)
xg (250, 100, 100, 11, 2)
xw (250, 2, 5)
xc (250, 10, 5)
xt (250, 10, 1)


In [605]:
def make_MiONet(n_ch:int=128):
    
    input_m = Input(shape=(nx,ny,nz,2))
    input_g = Input(shape=(nx,ny,nz,2))
    input_w = Input(shape=(2,5))
    input_c = Input(shape=(None,5))
    input_t = Input(shape=(None,1))

    def conv_block(ch, inp, pool_size=(2,2,2)):
        _ = Conv3D(ch, (3,3,1), padding='same')(inp)
        _ = Conv3D(ch, (1,1,3), padding='same')(_)
        _ = GroupNormalization(groups=-1)(_)
        _ = MaxPooling3D(pool_size)(_)
        return gelu(_)
    
    def time_deconv_block(ch, inp, strides=(2,2,2)):
        _ = TimeDistributed(Conv3DTranspose(ch, 3, strides=strides, padding='same'))(inp)
        _ = ConvLSTM3D(ch, 3, padding='same', return_sequences=True)(_)
        _ = GroupNormalization(groups=-1)(_)
        return gelu(_)
    
    def dense_block(ch, inp):
        _ = Dense(ch)(inp)
        _ = BatchNormalization()(_)
        return gelu(_)

    # controls
    c = dense_block(n_ch//4, input_c)
    c = dense_block(n_ch//2, c)
    c = dense_block(n_ch, c)

    # locations
    w = dense_block(n_ch//4, input_w)
    w = dense_block(n_ch//2, w)
    w = dense_block(n_ch, w)

    # time
    t = dense_block(n_ch//4, input_t)
    t = dense_block(n_ch//2, t)
    t = dense_block(n_ch, t)

    # model
    m = conv_block(16, input_m)
    m = conv_block(128, m)
    _, h_, w_, d_, c_ = m.shape
    m = Reshape((h_*w_*d_,c_))(m)

    # grid
    g = conv_block(16, input_g)
    g = conv_block(128, g)
    _, h_, w_, d_, c_ = g.shape
    g = Reshape((h_*w_*d_,c_))(g)

    # merge
    mg = keras.ops.einsum('bpc,bpc->bpc', m, g)
    wc = keras.ops.einsum('bpc,bqc->bqc', w, c)
    xx = keras.ops.einsum('btz,bpc->btpc', t, mg)

    xx = Reshape((None, h_, w_, d_, c_))(xx)

    xx = time_deconv_block(128, xx, strides=(2,2,3))
    xx = time_deconv_block(16, xx)
    xx = TimeDistributed(Cropping3D(((0,0),(0,0),(0,1))))(xx)

    xx = ConvLSTM3D(2, 3, padding='same', return_sequences=True)(xx)
    
    return Model([input_m, input_g, input_w, input_c, input_t], xx)

In [577]:
model = make_MiONet()
model.compile(optimizer=AdamW(1e-3, 1e-5), loss='mse', metrics=['mse'])

history = model.fit(x=[xm,xg,xw,xc,xt], y=[],
                    epochs=30,
                    batch_size=8,
                    validation_split=0.2,
                    verbose=1)

***
# END